In [0]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
gold_erp_CUST_AZ12_df=spark.read.format("delta").load("/Volumes/etl/medal/silver/erp/silver_erp_CUST_AZ12")
gold_erp_PX_CAT_G1V2_df=spark.read.format("delta").load("/Volumes/etl/medal/silver/erp/silver_erp_LOC_A101")
gold_erp_LOC_A101_df=spark.read.format("delta").load("/Volumes/etl/medal/silver/erp/erp_PX_CAT_G1V2")
gold_crm_cust_df=spark.read.format("delta").load("/Volumes/etl/medal/silver/crm/crm_cust")
gold_crm_prd_info_df=spark.read.format("delta").load("/Volumes/etl/medal/silver/crm/crm_prd_info")
gold_crm_sales_details_df=spark.read.format("delta").load("/Volumes/etl/medal/silver/crm/crm_sales_details")

In [0]:
dim_prd=gold_crm_prd_info_df.join(gold_erp_LOC_A101_df,gold_crm_prd_info_df.category_id == gold_erp_LOC_A101_df.id, "left")
dim_prd= dim_prd.select(F.row_number().over(Window.orderBy("prd_id","prd_start_dt")).alias("product_key"),F.col("prd_id").alias("product_id"),F.col("category_id"),F.col("prd_key").alias("product_number"),F.col("prd_nm").alias("product_name"),F.col("category"),F.col("sub_category"),F.col("maintenance"),F.col("prd_cost").alias("product_cost"),F.col("prd_line").alias("product_line"),F.col("prd_start_dt").alias("product_start_date"))
display(dim_prd.limit(20))

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


product_key,product_id,category_id,product_number,product_name,category,sub_category,maintenance,product_cost,product_line,product_start_date
1,210,CO_RF,FR-R92B-58,HL Road Frame - Black- 58,Components,Road Frames,Yes,0,Road,2003-07-01
2,211,CO_RF,FR-R92R-58,HL Road Frame - Red- 58,Components,Road Frames,Yes,0,Road,2003-07-01
3,212,AC_HE,HL-U509-R,Sport-100 Helmet- Red,Accessories,Helmets,Yes,12,Other Sales,2011-07-01
4,213,AC_HE,HL-U509-R,Sport-100 Helmet- Red,Accessories,Helmets,Yes,14,Other Sales,2012-07-01
5,214,AC_HE,HL-U509-R,Sport-100 Helmet- Red,Accessories,Helmets,Yes,13,Other Sales,2013-07-01
6,215,AC_HE,HL-U509,Sport-100 Helmet- Black,Accessories,Helmets,Yes,12,Other Sales,2011-07-01
7,216,AC_HE,HL-U509,Sport-100 Helmet- Black,Accessories,Helmets,Yes,14,Other Sales,2012-07-01
8,217,AC_HE,HL-U509,Sport-100 Helmet- Black,Accessories,Helmets,Yes,13,Other Sales,2013-07-01
9,218,CL_SO,SO-B909-M,Mountain Bike Socks- M,Clothing,Socks,No,3,Mountain,2011-07-01
10,219,CL_SO,SO-B909-L,Mountain Bike Socks- L,Clothing,Socks,No,3,Mountain,2011-07-01


In [0]:
dim_prd.write.mode("overwrite").format("delta")\
.save("/Volumes/etl/medal/gold/dim_products")

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
dim_cust=gold_crm_cust_df.join(gold_erp_CUST_AZ12_df, gold_crm_cust_df.cst_key == gold_erp_CUST_AZ12_df.customer_id, "left").join(gold_erp_PX_CAT_G1V2_df, gold_crm_cust_df.cst_key == gold_erp_PX_CAT_G1V2_df.customer_id, "left")
dim_cust= dim_cust.select(F.row_number().over(Window.orderBy(F.col("cst_create_date"))).alias("customer_key"),F.col("cst_id").alias("customer_id"),F.col("cst_key").alias("customer_number"),F.col("cst_firstname").alias("customer_firstname"),F.col("cst_lastname").alias("customer_lastname"),F.col("cst_marital_status").alias("customer_marital_status"),
 F.when(F.col("cst_gndr") =="N/A",F.upper(F.col("gender"))).otherwise(F.col("cst_gndr")).alias("customer_gender"),F.col("cst_create_date").alias("customer_create_date"),F.col("birth_date"),F.col("country"))
display(dim_cust.limit(20))

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


customer_key,customer_id,customer_number,customer_firstname,customer_lastname,customer_marital_status,customer_gender,customer_create_date,birth_date,country
1,11014,AW00011014,Sydney,Bennett,SINGLE,FEMALE,2025-10-06,1973-11-06,US
2,11011,AW00011011,Curtis,Lu,MARRIED,MALE,2025-10-06,1969-05-03,Australia
3,11002,AW00011002,Ruben,Torres,MARRIED,MALE,2025-10-06,1971-02-09,Australia
4,11000,AW00011000,Jon,Yang,MARRIED,MALE,2025-10-06,1971-10-06,Australia
5,11001,AW00011001,Eugene,Huang,SINGLE,MALE,2025-10-06,1976-05-10,Australia
6,11004,AW00011004,Elizabeth,Johnson,SINGLE,FEMALE,2025-10-06,1979-08-05,Australia
7,11007,AW00011007,Marco,Mehta,MARRIED,MALE,2025-10-06,1969-11-06,Australia
8,11006,AW00011006,Janet,Alvarez,SINGLE,FEMALE,2025-10-06,1976-12-02,Australia
9,11008,AW00011008,Rob,Verhoff,SINGLE,FEMALE,2025-10-06,1975-07-04,Australia
10,11015,AW00011015,Chloe,Young,SINGLE,FEMALE,2025-10-06,1984-08-26,US


In [0]:
dim_cust.write.mode("overwrite").format("delta")\
.save("/Volumes/etl/medal/gold/dim_customer")

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
fact_sales=gold_crm_sales_details_df.join(dim_prd,gold_crm_sales_details_df.sls_prd_key == dim_prd.product_number, "left").join(dim_cust,gold_crm_sales_details_df.sls_cust_id == dim_cust.customer_id, "left")
fact_sales= fact_sales.select(  F.col("sls_ord_num").alias("order_number"),
                  F.col("product_key").alias("product_key"),
                      F.col("customer_key").alias("customer_key"), 
    F.col("sls_order_dt").alias("order_date"),
    F.col("sls_ship_dt").alias("ship_date"),
    F.col("sls_due_dt").alias("due_date"),
    F.col("sls_price").alias("sls_price"),
    F.col("sls_quantity").alias("quantity"),
    F.col("sls_sales").alias("sales_amount"),
    

 
    )
display(fact_sales.limit(20))

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


order_number,product_key,customer_key,order_date,ship_date,due_date,sls_price,quantity,sales_amount
SO43697,101,10769,2010-12-29,2011-01-05,2011-01-10,3578.0,1,3578
SO43698,137,17390,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400
SO43699,137,14864,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400
SO43700,128,3502,2010-12-29,2011-01-05,2011-01-10,699.0,1,699
SO43701,137,4,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400
SO43702,102,16646,2010-12-30,2011-01-06,2011-01-11,3578.0,1,3578
SO43703,101,5625,2010-12-30,2011-01-06,2011-01-11,3578.0,1,3578
SO43704,142,6,2010-12-30,2011-01-06,2011-01-11,3375.0,1,3375
SO43705,135,12,2010-12-30,2011-01-06,2011-01-11,3400.0,1,3400
SO43706,103,16622,2010-12-31,2011-01-07,2011-01-12,3578.0,1,3578


In [0]:
fact_sales.write.mode("overwrite").format("delta")\
.save("/Volumes/etl/medal/gold/fact_sales")

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(
